In [1]:
!pip install dagshub
!pip install mlflow
!pip install pyngrok

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.9/247.9 kB 5.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 68.2 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 3.6 MB/s eta 0:00:00
  Created wheel for fusepy: filename=fusepy-3.0.1-py3-none-any.whl size=10487 sha256=84d8362bf5e4915429942a5cbe0427652a39d60877ff1659ce9971e90244aaad
  Stored in directory: /root/.cache/pip/wheels/c0/18/f6/f0d6be9d0435e2677ce5cc758e91da50053dce456a346f08c5
Successfully built fusepy
  Attempting uninstall: dacite
    Found existing installation: dacite 1.8.1
    Uninstalling dacite-1.8.1:
      Successfully uninstalled dacite-1.8.1
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalli

In [23]:
import dagshub
import mlflow
import os
import time
import numpy as np
import xgboost as xgb
import h2o
import glob
import gc
from pyngrok import ngrok
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from sklearn.metrics import roc_auc_score,precision_score,recall_score,f1_score,accuracy_score

In [3]:
os.environ['MLFLOW_TRACKING_USERNAME'] = 'tnbmarketplace'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'd7c1a451bfc46bf96fbbe5d723becf3955dfa2b3'
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/tnbmarketplace/instacart_basket_analysis_exp_tracking.mlflow'
!dagshub login --token "d7c1a451bfc46bf96fbbe5d723becf3955dfa2b3"

2024-10-17 05:34:47,240 - [INFO]  - [_client.py:1026] - HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"
✅ Token added successfully


In [4]:
dagshub.init(repo_name = "instacart_basket_analysis_exp_tracking",repo_owner = 'tnbmarketplace')

Accessing as tnbmarketplace

Initialized MLflow to track repo "tnbmarketplace/instacart_basket_analysis_exp_tracking"

Repository tnbmarketplace/instacart_basket_analysis_exp_tracking initialized!

In [7]:
!ngrok authtoken "2kvaYw5ZiG5bL8iM8YJBVJPk1Ru_3C16mMgmpKEBYb28PPLUe"
# Start Ngrok tunnel on H2O's port
ngrok_tunnel = ngrok.connect(54321)
print("H2O UI accessible at:", ngrok_tunnel)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                
H2O UI accessible at: NgrokTunnel: "https://c0c9-34-74-97-91.ngrok-free.app" -> "http://localhost:54321"


In [8]:
h2o.init(max_mem_size = 25)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu320.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu320.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpne4sn6kd
  JVM stdout: /tmp/tmpne4sn6kd/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpne4sn6kd/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 17 days
H2O_cluster_name:,H2O_from_python_unknownUser_ax9c15
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,25 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [9]:
def get_time(start):
    return time.time() - start

In [10]:
exp_data = h2o.import_file('/kaggle/input/instacart-model-patcher/experiment_data')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
# exp_data['fold_no'] = exp_data['fold_no'].asfactor()
exp_data['reordered'] = exp_data['reordered'].asfactor() 

In [13]:
training_cols = exp_data.columns
training_cols.remove('reordered')
training_cols.remove('fold_no')

In [14]:
# logistic_model = H2OGeneralizedLinearEstimator(family='binomial')
# logistic_model.train(x=training_cols,y='reordered',training_frame = exp_data,fold_column='fold_no')

In [15]:
train_data = exp_data[exp_data['fold_no'] != 2]
test_data = exp_data[exp_data['fold_no'] == 2]

In [17]:
reordered_array = test_data['reordered'].as_data_frame()
r_array = train_data['reordered'].as_data_frame()

/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [18]:
training_cols = train_data.columns
training_cols.remove('reordered')
training_cols.remove('fold_no')

In [27]:
class ModelTrainer:
    
    def __init__(self,experiment_name,train_set,test_set=None):
        self.train_set = train_set
        self.test_set = test_set
        self.exp_name = experiment_name
        mlflow.set_experiment(self.exp_name)
        
    def train_h2oglm(self,params=None,prev_commit_hash = None):
        h2o_logistic_model = H2OGeneralizedLinearEstimator(family='binomial') \
                            .train(x=train_set.drop("reordered").columns,y='reordered',training_frame=train_set)
        with mlflow.start_run():
            mlflow.h2o.log_model(h2o_logistic_model,"h2o_logistic_model")
            # log perameters
            mlflow.log_param("family","binomial")
            if params != None:
                for key,value in params:
                    mlflow.log_param(key,value)
            else:
                mlflow.log_param("alpha",h2o_logistic_model.alpha[0])
                mlflow.log_param("lambda",h2o_logistic_model.lambda_[0])
                
            if self.test_set != None:  
                
                perf = h2o_logistic_model.model_performance(test_data = self.test_set)
                default_threshold_index = 0  # This is for 0.5 threshold
                best_f1_threshold_index = max(range(len(perf.F1())), key=lambda i: perf.F1()[i][1])
                best_rec_threshold_index = max(range(len(perf.recall())), key=lambda i: perf.recall()[i][1])
                best_prec_threshold_index = max(range(len(perf.precision())), key=lambda i: perf.precision()[i][1])
                best_acc_threshold_index = max(range(len(perf.accuracy())), key=lambda i: perf.accuracy()[i][1])
                thresholds = [default_threshold_index,best_f1_threshold_index,best_rec_threshold_index,best_prec_threshold_index,best_acc_threshold_index]
                
                for i, threshold in enumerate(thresholds):
                    
                    precision = perf.precision()[i][1]
                    recall = perf.recall()[i][1]
                    f1_score = perf.F1()[i][1]
                    accuracy = perf.accuracy()[i][1]

                    # Log metrics with threshold as a tag or suffix
                    mlflow.log_metric(f"Precision_at_threshold_{threshold:.2f}", precision)
                    mlflow.log_metric(f"Recall_at_threshold_{threshold:.2f}", recall)
                    mlflow.log_metric(f"F1_at_threshold_{threshold:.2f}", f1_score)
                    mlflow.log_metric(f"Accuracy_at_threshold_{threshold:.2f}", accuracy)    
                
                #log metrics 
                mlflow.log_metric("AUC",perf.auc())
                mlflow.log_metric("logloss",perf.logloss())
            

In [ ]:
preds = logistic_model.predict(test_data.drop(['fold_no','reordered'],axis=1))[2]
preds_df = preds.as_data_frame()
prediction = preds_df['p1'].tolist()